In [32]:
from neo4j import GraphDatabase
import geopandas as gpd

shp = r"C:\Users\adria\Desktop\STUDIA_FOLDERY\wyznaczanie-trasy\skjz\direction_calosc_0.shp"
creds = r"C:\Users\adria\Desktop\STUDIA_FOLDERY\wyznaczanie-trasy\neo4j\local_credentials.txt"

In [25]:
road_classes_speed = {
    "A": 140,
    "S": 120,
    "GP": 100,
    "G": 80,
    "Z": 60,
    "L": 50,
    "D": 40,
    "I": 30,
}

for key in road_classes_speed:
    road_classes_speed[key] /= 2

# połączenie z lokalną bazą

In [26]:
with open (creds, "r") as file:
    lines = file.readlines()
    uri = lines[0].strip()
    user = lines[1].strip()
    password = lines[2].strip()

driver = GraphDatabase.driver(uri, auth=(user, password))

# załadowanie nodów i relacji do bazy

In [27]:
# gdf = gpd.read_file(shp)

# with driver.session() as session:
#     for idx,row in gdf.iterrows():
#         length = row.geometry.length
#         coords = list(row.geometry.coords)
#         startPoint = coords[0]
#         endPoint = coords[-1]
#         x1 = startPoint[0]
#         y1 = startPoint[1]
#         x2 = endPoint[0]
#         y2 = endPoint[1]
        
#         edge_id = idx
#         length = row.geometry.length
#         road_class_speed = road_classes_speed[row['klasaDrogi']]
#         time = length/(road_class_speed * 1000 / 3600)
#         direction = row['kierunek']
        
#         query = f"""
#         MERGE (n:Node {{id: apoc.util.md5([{int(x1)}, {int(y1)}]), location: point({{x: {x1}, y: {y1}}})}})
#         MERGE (m:Node {{id: apoc.util.md5([{int(x2)}, {int(y2)}]), location: point({{x: {x2}, y: {y2}}})}})
#         """
#         if direction == 0:
#             query += f"""
#             MERGE (n)-[:CONNECTION {{edge_id: {edge_id}, length: {length}, time: {time}}}]->(m)
#             MERGE (m)-[:CONNECTION {{edge_id: {edge_id}, length: {length}, time: {time}}}]->(n)
#             RETURN n, m;
#             """
#         session.run(query)

# wybranie losowych punktów do testu

In [28]:
gdf = gpd.read_file(shp)

punkty = []
for idx,row in gdf.iterrows():
    coords = list(row.geometry.coords)
    startPoint = coords[0]
    endPoint = coords[-1]
    x1 = startPoint[0]
    y1 = startPoint[1]
    x2 = endPoint[0]
    y2 = endPoint[1]
    punkty.append([x1,y1])
    punkty.append([x2,y2])

import random
first_random_point = random.choice(punkty)
second_random_point = random.choice(punkty)

print("Punkt 1: ", int(first_random_point[0]), int(first_random_point[1]))
print("Punkt 2: ", int(second_random_point[0]), int(second_random_point[1]))

Punkt 1:  475451 570280
Punkt 2:  472896 571858


# załadowanie grafa

In [ ]:
# query = """ 
# CALL gds.graph.project(
#   'NavigationGraph',              
#   'Node',                       
#   {
#     CONNECTION: {                         
#       properties: ['length', 'time']      
#     }
#   }
# )
# """

# with driver.session() as session:
#     result = session.run(query)

In [29]:
import json
def save_geojson(driver, query, file_path):
    with driver.session() as session:
        result = session.run(query)
        geojson = result.single()['geojson']
        with open(file_path, 'w') as f:
            json.dump(geojson, f, indent=4)

# najkrótsza ścieżka

In [30]:
cypher_query = """
MATCH (start:Node {id : apoc.util.md5([471553, 570358])})
MATCH (end:Node {id : apoc.util.md5([473004, 574156])})
CALL gds.shortestPath.dijkstra.stream('NavigationGraph', {
  sourceNode: start,
  targetNode: end,
  relationshipWeightProperty: 'length' 
})
YIELD path
WITH nodes(path) as nodes
RETURN {
  type: "FeatureCollection",
  crs: {
    type: "name",
    properties: {
      name: "EPSG:2180"
    }
  },
  features: [
    {
      type: "Feature",
      geometry: {
        type: "LineString",
        coordinates: [node IN nodes | [node.location.x, node.location.y]]
      },
      properties: {
        name: "Shortest Path",
        totalNodes: size(nodes)
      }
    }
  ]
} AS geojson
"""

save_geojson(driver, cypher_query, 'shortest_path.geojson')


# najszybsza ścieżka

In [31]:
cypher_query = """
MATCH (start:Node {id : apoc.util.md5([471553, 570358])})
MATCH (end:Node {id : apoc.util.md5([473004, 574156])})
CALL gds.shortestPath.dijkstra.stream('NavigationGraph', {
  sourceNode: start,
  targetNode: end,
  relationshipWeightProperty: 'time' 
})
YIELD path
WITH nodes(path) as nodes
RETURN {
  type: "FeatureCollection",
  crs: {
    type: "name",
    properties: {
      name: "EPSG:2180"
    }
  },
  features: [
    {
      type: "Feature",
      geometry: {
        type: "LineString",
        coordinates: [node IN nodes | [node.location.x, node.location.y]]
      },
      properties: {
        name: "Shortest Path",
        totalNodes: size(nodes)
      }
    }
  ]
} AS geojson
"""

save_geojson(driver, cypher_query, 'quickest_path.geojson')